In [1]:
!pip install psycopg2-binary pandas sqlalchemy

In [2]:
from sqlalchemy import create_engine
import pandas as pd 

db_user = 'postgres'
db_password = '17032004'
db_host = 'localhost'
db_port = '5432'
db_name = 'customer_segmentation_db'


engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


In [9]:
##### oct 2019
query = """ SELECT event_type, COUNT(*) as total
        FROM oct_clickstream_2019
        GROUP BY event_type
        ORDER BY total DESC;"""

df_event_summary = pd.read_sql(query, engine)
df_event_summary.head()


print(df_event_summary)

df_event_summary.to_csv("event_type_summary_oct_2019.csv", index=False)

  event_type     total
0       view  40779399
1       cart    926514
2   purchase    742849


In [10]:
#### nov 2019
query = """ SELECT event_type, COUNT(*) as total
        FROM nov_clickstream_2019
        GROUP BY event_type
        ORDER BY total DESC;"""

df_event_summary = pd.read_sql(query, engine)
df_event_summary.head()


print(df_event_summary)

df_event_summary.to_csv("event_type_summary_nov_2019.csv", index=False)

  event_type     total
0       view  63556110
1       cart   3028920
2   purchase    916939


## moving to RFM 

In [11]:
query = """SELECT 
user_id,
price ,
event_time:: date as purchase_date
FROM oct_clickstream_2019
WHERE event_type = 'purchase'
AND user_id IS NOT NULL
AND price IS NOT NULL
UNION ALL
SELECT 
user_id,
price,
event_time::date as purchase_date
FROM nov_clickstream_2019
WHERE event_type = 'purchase'
AND user_id IS NOT NULL
AND price IS NOT NULL;"""
df_purchase = pd.read_sql(query, engine)
df_purchase.head()


,user_id,price,purchase_date
0,543272936,130.76,2019-10-01
1,551377651,642.69,2019-10-01
2,514591159,29.51,2019-10-01
3,555332717,54.42,2019-10-01
4,524601178,189.91,2019-10-01


In [15]:
#### RFM calculation 
import pandas as pd 
from datetime import datetime 

df_purchase['purchase_date'] = pd.to_datetime(df_purchase['purchase_date'])

reference_date = df_purchase['purchase_date'].max() +pd.Timedelta(days=1)

rfm = df_purchase.groupby('user_id').agg({
    'purchase_date': lambda x: (reference_date - x.max()).days, #####  recency 
    'user_id': 'count',              ##### frequency
    'price': 'sum'  ###### monetary
}).rename(columns ={ 'purchase_date': 'Recency',
                     'user_id': 'Frequency',
                     'price': 'Monetary'}).reset_index()
rfm.head()


,user_id,Recency,Frequency,Monetary
0,138340325,20,1,93.50
1,225644257,19,1,40.91
2,253299396,25,1,246.85
3,256164170,11,1,113.23
4,264649825,56,2,1240.04


In [16]:
##### RFM scoring for better understading 
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1]).astype(int)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5]).astype(int)

rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].sum(axis=1)

rfm.head()


,user_id,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment,RFM_Score
0,138340325,20,1,93.50,3,1,2,312,6
1,225644257,19,1,40.91,3,1,1,311,5
2,253299396,25,1,246.85,3,1,3,313,7
3,256164170,11,1,113.23,5,1,2,512,8
4,264649825,56,2,1240.04,1,3,5,135,9


In [21]:
#### Assigning customer segmentation for better insights 
def assign_segment(row):
    if row ['RFM_Score'] >= 13:
        return 'VIP'
    elif row['R_Score'] >= 4 and row ['F_Score'] >= 4:
        return 'Loyal'
    elif row['R_Score'] <= 2  and row['F_Score'] > 3:
        return 'At Risk'
    elif row['R_Score'] == 5 and row['F_Score'] == 1:
        return 'New Customer'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(assign_segment, axis=1)
rfm.head(10)

rfm.to_csv("rfm_customer_segmentation.csv", index=False)

In [22]:
segment_counts = rfm['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Count']
segment_counts


,Segment,Count
0,Others,433803
1,VIP,111562
2,At Risk,83924
3,Loyal,50333
4,New Customer,17848


In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

df = pd.read_csv("mba_purchases.csv", parse_dates=["event_time"])

df['basket_id'] = df['user_session'].fillna(
    df['user_id'].astype(str) + "_" + df['event_time'].dt.date.astype(str))

product_counts = df['product_id'].value_counts()
rare_products = product_counts[product_counts < 5].index
df = df[~df['product_id'].isin(rare_products)]

basket = df.groupby(['basket_id', 'product_id'])['product_id'].count().unstack().fillna(0)


basket = basket.astype(bool)


frequent_itemsets = apriori(basket, min_support=0.0005, use_colnames=True)


rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)


rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join([str(i) for i in list(x)]))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join([str(i) for i in list(x)]))


rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

rules.to_csv("mba_results.csv", index=False)
print("MBA complete! Rules exported to mba_results.csv")


MBA complete! Rules exported to mba_results.csv


In [ ]:
print("Unique Baskets:", df['basket_id'].nunique())
print("Unique Products:", df['product_id'].nunique())
print("Total Records:", len(df))



In [9]:
from datetime import datetime
import pandas as pd 

df = pd.read_csv("CustomerSegments.csv")
today = datetime.today().strftime('%Y-%m-%d')

segment_counts = df['segment'].value_counts().to_dict()

# Build prompt
segment_text = "\n".join([f"- {k}: {v}" for k, v in segment_counts.items()])
rfm_prompt = f"""
You are a senior data analyst. Today is {today}.

You are analyzing customer behavior based on the following RFM segments:
{segment_text}

Based on this:
1. Give 3–5 business insights
2. Suggest 2–3 strategies for improvement
3. Prioritize segments to focus on

Respond like a BI analyst writing a business summary.
"""
print(rfm_prompt)



You are a senior data analyst. Today is 2025-06-21.

You are analyzing customer behavior based on the following RFM segments:
- Others: 606032
- New Customer: 54438
- At Risk: 16094
- VIP: 11237
- Loyal: 9669

Based on this:
1. Give 3–5 business insights
2. Suggest 2–3 strategies for improvement
3. Prioritize segments to focus on

Respond like a BI analyst writing a business summary.

